## Assignment:
* Predicting House Prices Using a Feedforward Neural Network

### Objective:
* Create a feedforward neural network to predict house prices based on various features such as the number of rooms, location, size, and other factors.

#### Dataset:
* Use the "House Prices: Advanced Regression Techniques" dataset available on Kaggle. You can download the dataset from this link: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

### Tasks:
1. Define the problem: Determine the goal of the project and the problem you're trying to solve using the feedforward neural network.

2. Gather the data: Download the dataset from the Kaggle link provided above.

3. Prepare the data: Clean and preprocess the data. Handle missing values, encode categorical variables, and normalize or standardize numerical variables.

4. Create the model: Build a feedforward neural network using Keras with an appropriate architecture for predicting house prices. You can experiment with different numbers of layers, neurons, and activation functions.

5. Train the model: Train the neural network on the preprocessed data. Experiment with different batch sizes and numbers of epochs to improve the model's performance.

6. Evaluate the model: Test the model's performance on a separate validation or test set. Calculate metrics such as mean squared error, mean absolute error, and R-squared to assess the model's accuracy.

7. Optimize the model: Perform hyperparameter tuning to find the optimal architecture and parameters for the neural network. You can use techniques like Grid Search or Random Search.

8. Present your findings: Create a report or presentation summarizing your methodology, model architecture, performance metrics, and any insights or recommendations based on your findings.

* This assignment will help you gain a deeper understanding of feedforward neural networks and their applications in regression tasks. Additionally, you'll gain experience in data preprocessing, model evaluation, and hyperparameter tuning.

# Problem Statement
> The goal of this project is to create a feedforward neural network to predict house prices based on various features
such as the number of rooms, location, size, and other factors. Developing a model that can accurately predict the
price of a house given its features.

In [14]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [15]:
# Load the dataset
from google.colab import drive
drive.mount('/content/gdrive')
train_data = pd.read_csv("/content/gdrive/My Drive/train.csv")
test_data = pd.read_csv("/content/gdrive/My Drive/test.csv")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [16]:
train_data.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


In [17]:
# Preprocess the data
train_data.fillna(-1, inplace=True)
test_data.fillna(-1, inplace=True)

In [18]:
train_data.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,-1,Reg,Lvl,AllPub,...,0,-1,-1,-1,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,-1,Reg,Lvl,AllPub,...,0,-1,-1,-1,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,-1,IR1,Lvl,AllPub,...,0,-1,-1,-1,0,9,2008,WD,Normal,223500


In [19]:
# Convert categorical features to strings
cat_cols = train_data.select_dtypes(include=['object']).columns
train_data[cat_cols] = train_data[cat_cols].astype(str)
test_data[cat_cols] = test_data[cat_cols].astype(str)

In [20]:
# Encode categorical features
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features_train = one_hot_encoder.fit_transform(train_data[cat_cols]).toarray()
encoded_features_test = one_hot_encoder.transform(test_data[cat_cols]).toarray()

In [21]:
# Combine encoded features with numeric features
numeric_cols = train_data.select_dtypes(include=['int64', 'float64']).drop(['Id', 'SalePrice'], axis=1).columns
X_train = pd.concat([pd.DataFrame(encoded_features_train), train_data[numeric_cols].reset_index(drop=True)], axis=1)
X_test = pd.concat([pd.DataFrame(encoded_features_test), test_data[numeric_cols].reset_index(drop=True)], axis=1)
y_train = train_data['SalePrice']

In [22]:
# Convert all of the feature names to strings.
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [23]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
# Create the model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(1, activation="linear"))
model.compile(optimizer='adam', loss='mse')

In [25]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Predict on test data
y_pred = model.predict(X_test)

# Optional: evaluate the model using a separate validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
model.fit(X_train, y_train, epochs=100, batch_size=32)
y_val_pred = model.predict(X_val)

mse = mean_squared_error(y_val, y_val_pred)
mae = mean_absolute_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)

print(f"Mean Squared Error: {mse}, Mean Absolute Error: {mae}, R-squared: {r2}")

Epoch 1/100
46/46 [==============================] - 1s 3ms/step - loss: 39035232256.0000
Epoch 2/100
46/46 [==============================] - 0s 4ms/step - loss: 38928932864.0000
Epoch 3/100
46/46 [==============================] - 0s 4ms/step - loss: 37929635840.0000
Epoch 4/100
46/46 [==============================] - 0s 4ms/step - loss: 33641904128.0000
Epoch 5/100
46/46 [==============================] - 0s 4ms/step - loss: 23657449472.0000
Epoch 6/100
46/46 [==============================] - 0s 4ms/step - loss: 11772857344.0000
Epoch 7/100
46/46 [==============================] - 0s 4ms/step - loss: 6295247360.0000
Epoch 8/100
46/46 [==============================] - 0s 4ms/step - loss: 4399073792.0000
Epoch 9/100
46/46 [==============================] - 0s 4ms/step - loss: 3260942080.0000
Epoch 10/100
46/46 [==============================] - 0s 4ms/step - loss: 2602403584.0000
Epoch 11/100
46/46 [==============================] - 0s 4ms/step - loss: 2239710464.0000
Epoch 12/100
